In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import random
import csv
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import pandas as pd
import numpy as np
#import QSTK.qstkfeat.featutil as ftu
from pyalgotrade import strategy
from pyalgotrade import barfeed
from pyalgotrade import bar
from pyalgotrade.technical import ma as pytechma
from pyalgotrade import plotter
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import sharpe
from pyalgotrade.utils import stats as pytechstats
import datetime as dt
import os

In [4]:
download_dir = '/Users/shadmanspc/Desktop/ADA/assign2'
# training data url for time 1 Apr, 2012 to 31 Mar, 2016
#download_dir = 'C:\Users\user\Desktop\stck/'
# training data url for time 1 Apr, 2012 to 31 Mar, 2016
training_data_url = 'http://real-chart.finance.yahoo.com/table.csv?s=%s&a=03&b=1&c=2012&d=02&e=31&f=2016&g=d&ignore=.csv'
# stocks used as training data, all stocks from 0060.HK to 0100.HK
# change start and end to manipulate the data set
start_stock = 1
end_stock = 1
stocks = []

for counter in range(start_stock, end_stock + 1):
    stocks.append('%04d.HK' % counter)



# keys from yahoo are different from what qstk library expects
yahoo_keys = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
qstk_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']

raw_training_data = {}
# cloned for iteration, will remove those stocks from master stocks list if
# issue found during downloading data
stocks_clone = [s for s in stocks]
for s in stocks_clone:
    file_name = download_dir + "/"+ s + '_training'+'.csv'
    print(file_name)
    if(os.path.isfile(file_name)) :
        raw_training_data[s] = pd.read_csv(file_name)
    else:
        stock_url = training_data_url % s
        try:
            raw_training_data[s] = pd.read_csv(stock_url)
            raw_training_data[s].to_csv(file_name, sep=',')
        except (urllib2.HTTPError, urllib2.URLError) as ex:
            print ('Not found %s stock when downloading training data' %s)
            if s in stocks:
                stocks.remove(s)
            pass

def sanitizedataforqstk(frame):
    data = {}
    for k in yahoo_keys:
        key_df = pd.DataFrame()
        for s in stocks:
            dates = np.array([dt.datetime.strptime(t, "%Y-%m-%d") for t in frame[s]['Date']])
            key_df[s] = pd.Series(data=frame[s][k].values, index=dates).sort_index(ascending=True)
        data[k] = key_df

    # sanitize to use the earlier keys
    data[qstk_keys[0]] = data[yahoo_keys[0]]
    data[qstk_keys[1]] = data[yahoo_keys[1]]
    data[qstk_keys[2]] = data[yahoo_keys[2]]
    data[qstk_keys[3]] = data[yahoo_keys[3]]
    data[qstk_keys[4]] = data[yahoo_keys[4]]
    data[qstk_keys[5]] = data[yahoo_keys[5]]
    del data[yahoo_keys[0]]
    del data[yahoo_keys[1]]
    del data[yahoo_keys[2]]
    del data[yahoo_keys[3]]
    del data[yahoo_keys[4]]
    del data[yahoo_keys[5]]
    for key in qstk_keys:
        data[key] = data[key].fillna(method='ffill')
        data[key] = data[key].fillna(method='bfill')
        data[key] = data[key].fillna(1.0)

    return data

training_data = sanitizedataforqstk(raw_training_data)
print(training_data)

/Users/shadmanspc/Desktop/ADA/assign2/0001.HK_training.csv
{'open':                0001.HK
2018-09-24  203.259995
2018-09-25  202.199997
2018-09-26  203.960007
2018-09-27  201.029999
2018-09-28  201.429993
...                ...
2019-08-26  208.479996
2019-08-27  211.720001
2019-08-28  211.949997
2019-08-29  211.000000
2019-08-30  215.889999

[237 rows x 1 columns], 'high':                0001.HK
2018-09-24  203.259995
2018-09-25  203.789993
2018-09-26  203.960007
2018-09-27  204.000000
2018-09-28  203.139999
...                ...
2019-08-26  211.399994
2019-08-27  212.229996
2019-08-28  213.860001
2019-08-29  213.250000
2019-08-30  219.449997

[237 rows x 1 columns], 'low':                0001.HK
2018-09-24  200.619995
2018-09-25  200.720001
2018-09-26  200.889999
2018-09-27  200.309998
2018-09-28  200.250000
...                ...
2019-08-26  207.720001
2019-08-27  210.514999
2019-08-28  211.000000
2019-08-29  209.660004
2019-08-30  214.160004

[237 rows x 1 columns], 'close':      

In [5]:
def featMA( dData, lLookback=30, bRel=True, b_human=False ):
    '''
    @summary: Calculate moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    dfRet = dfPrice.rolling(lLookback).mean()
    #ts_log.rolling(12).mean()
    
    if bRel:
        dfRet = dfRet / dfPrice
    if b_human:  
        data2 = dfRet * dData['close']
        data3 = pand.DataFrame({"Raw":data2[data2.columns[0]]})
        for sym in dfRet.columns:
            if sym != '$SPX' and sym != '$VIX':
                data3[sym + " Moving Average"] = data2[sym]
                data3[sym] = dData['close'][sym]
        del data3['Raw']
        return data3
    return dfRet

ma_training=featMA(training_data,bRel=False).fillna(0.0)

In [6]:
def returnize0(nds):
    """
    @summary Computes stepwise (usually daily) returns relative to 0, where
    0 implies no change in value.
    @return the array is revised in place
    """
    s= np.shape(nds)
    if len(s)==1:
        nds=np.expand_dims(nds,1)
    #print(s)
    nds[1:, :] = (nds[1:, :] - nds[0:-1]) / abs(nds[0:-1])
    nds[0, :] = np.zeros(nds.shape[1])
    
    #return nds

def featMomentum(dData, lLookback=20, b_human=False ):
    '''
    @summary: N day cumulative return (based on 1) indicator
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    dfPrice = dData['close'].copy()
    #print(dfPrice.values)
    
    #Calculate Returns
    returnize0(dfPrice.values)
    
    #Calculate rolling sum
    dfRet = dfPrice.rolling(lLookback).sum()
    #print(dfRet.head(100))
    
    
    return dfRet

mom_training=featMomentum(training_data).fillna(0.0)
print(mom_training.head(100))

             0001.HK
2018-09-24  0.000000
2018-09-25  0.000000
2018-09-26  0.000000
2018-09-27  0.000000
2018-09-28  0.000000
...              ...
2019-02-08  0.074845
2019-02-11  0.072411
2019-02-12  0.072334
2019-02-13  0.045856
2019-02-14  0.056200

[100 rows x 1 columns]


In [7]:
def featRSI( dData, lLookback=14,  b_human=False):
    '''
    @summary: Calculate RSI
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past, 14 is standard
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''

    # create deltas per day
    delta = dData['close'].diff()
    #-----------
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(14).mean()
    RolDown = dDown.rolling(14).mean().abs()
    

    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

rsi_training=featRSI(training_data).fillna(0.0)
#print(rsi_training.head(100))

In [8]:
currentDataUrl = 'http://real-chart.finance.yahoo.com/table.csv?s=%s&a=%s&b=%s&c=%s&d=%s&e=%s&f=%s&g=d&ignore=.csv'
currentDate = dt.date.today()
oldDate = currentDate + dt.timedelta(days=-30)
raw_current_data = {}
for s in stocks_clone:
    file_name = download_dir + "/"+ s + '_current'+'.csv'
    print(file_name)
    if(os.path.isfile(file_name)) :
        raw_current_data[s] = pd.read_csv(file_name)
    else:    
        stock_url = currentDataUrl % (
            s, '%02d' % (oldDate.month - 1), oldDate.day, oldDate.year, '%02d' % (currentDate.month - 1), currentDate.day,
            currentDate.year)
        try:
            raw_current_data[s] = pd.read_csv(stock_url)
            raw_current_data[s].to_csv(file_name, sep=',')
        except (urllib2.HTTPError, urllib2.URLError) as ex:
            #print 'Not found %s stock when downloading current data' %s
            if s in stocks:
                stocks.remove(s)
            pass

current_data = sanitizedataforqstk(raw_current_data)

/Users/shadmanspc/Desktop/ADA/assign2/0001.HK_current.csv


In [9]:
toolbox = base.Toolbox()
# basic initialization for deap
def initDeap():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    # stochastic removed as High and Low can't be calculated for
    IND_SIZE = 3  
    # registerd coeff alias to coefficient function
    toolbox.register("coeffs", coefficient)
    # individual alias registerd to tools.initRepeat
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.coeffs, n=IND_SIZE)
    # population alias registered
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    # evaluate fucntion registered to computeFitness function
    toolbox.register("evaluate", compute_fitness)
    toolbox.register("mate", tools.cxSimulatedBinary, eta=0.3)
    toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=1.0, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=5)

# 1. rsi, 2. ma, 3. mom above is the order of coefficients removed pivot
# point and stochastic oscillator as one of the technical indicator, because
# lets say we are running this for the next 6 months,in that case we would
# not have the value of high, low of future. as such for future we can't use
# it.
def compute_fitness(individual):
    error = 0.0
    count = 0
    for key in stocks:
        # difference in RSI between day1 and day1 + 1
        rsidiff = rsi_training[key] - rsi_training[key].shift()
        # difference in moving average between day1 and day1 + 1
        madiff = ma_training[key] - ma_training[key].shift()
        # difference in momentum between day1 and day1 + 1
        momdiff = mom_training[key] - mom_training[key].shift()
        per_stock_count = 0
        dataframe_index = training_data[qstk_keys[3]][key].index.tolist()
        for (date, val) in rsi_training[key].iteritems():
            next_day = dataframe_index[per_stock_count + 1] if\
                per_stock_count <= len(dataframe_index) - 1 else None
            if not np.isnan(rsidiff[date]) and next_day is not None:
                #            Fitness defined as
                # market close (day2) - [ rsi difference between consecutive day * (coeff 1) +
                # moving average difference between consecutive day * (coeff 2) +
                # momentum difference between consecutive day * (coeff 3) +
                
                # Objective is to minimize the summation of squares of the above differences
                ft = (rsidiff[date]) * individual[0] + (madiff[date]) * individual[1] \
                     + (momdiff[date]) * individual[2]
                ft = ft + training_data[qstk_keys[3]][key][date]
                error = error + np.square(training_data[qstk_keys[3]][key][next_day] - ft)
                count = count + 1
                per_stock_count = per_stock_count + 1
    # dividing by total count gives the mean square error, which we want to minimize
    return (error / count,)


def coefficient():
    return random.random()

def predict(individual, days=1):
    prediction_series= []
    for d in list(range(days)):
        # calculate the rsi, ma and momentum
        # with each preddiction the underluing current_data dataframe is updated,
        # so we get the updated rsi for all the days
        rsi_current = featRSI(current_data).fillna(0.0)
        ma_current = featMA(current_data, bRel=False).fillna(0.0)
        mom_current = featMomentum(current_data).fillna(0.0)

        for key in stocks:
            # get the last valid date for which data is present
            last_date_with_data =\
                current_data[qstk_keys[3]][key].last_valid_index()
            next_date = last_date_with_data + dt.timedelta(days=1)
            # calculate the rsi,ma and momentum diff across consecutive days 
            rsi_current_diff = rsi_current[key] - rsi_current[key].shift()
            #print("rsi_diff")
            #print(rsi_current_diff)
            
            ma_current_diff = ma_current[key] - ma_current[key].shift()
            #print("ma_diff")
            #print(ma_current_diff)
            mom_current_diff = mom_current[key] - mom_current[key].shift()
            #print("mom_diff")
            #print(mom_current_diff)
            #  calculate the deviation based on the conefficients
            deviation = (rsi_current_diff.tail(1).values[0]) * individual[0]\
                + (ma_current_diff.tail(1).values[0]) * individual[1]+\
                (mom_current_diff.tail(1).values[0]) * individual[2]
            print(deviation)
            # predicted is addition of deviation + current close
            predicted_close_next_day = deviation + current_data[qstk_keys[3]][key][last_date_with_data]
            # update the current dataframe so we can continue calculating stocks
            current_data[qstk_keys[3]][key].set_value(next_date,predicted_close_next_day)
            
            prediction_series.append((key,next_date,predicted_close_next_day,
                                      current_data[qstk_keys[3]][key][last_date_with_data] ,
                                      deviation))
    
    return prediction_series




In [10]:
if __name__ == '__main__':
    initDeap()
    # population generated
    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("std", np.std)
    stats.register("median", np.median)
    # built in algo used with cross over probability = 0.5, mutation probability = 0.2 and number of generation = 5
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, stats=stats, halloffame=hof,
                                       verbose=True)
    print("Best individual is: %s\nwith fitness: %s" % (hof[0], hof[0].fitness))
    # this gives us the coefficients. Once we get the coefficients , we can use this coefficients,
    # we can use these to calculate the next day's value based on values on the present day
    
    # predict for 5 days, change the number of days to get that many days of prediction
    prediction_series = predict(hof[0],days=3)
    # sorting the prediction series with all entries of same stock together
   
    prediction_series_sorted = sorted(prediction_series, key=lambda tup: tup[0])
    print(prediction_series)
    filename = download_dir+'/results_predicted12.csv'
    with open(filename, 'w') as csvfile:
        fieldnames = ['stock',
                      'date','predicted_price','price_prev_day','deviation','movement']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for prediction in prediction_series_sorted:
            movement = 'DOWN' if prediction[4] < 0 else 'UP'
            writer.writerow(
                {'stock': prediction[0], 'date': prediction[1].strftime('%m/%d/%Y'),'predicted_price': prediction[2],'price_prev_day': prediction[3],'deviation':prediction[4],'movement':movement})
    


gen	nevals	avg    	min    	max    	std    	median 
0  	50    	70.8557	5.24258	179.055	41.1901	63.6673
1  	29    	45.713 	4.99853	321.197	70.3728	26.3387
2  	35    	15.6609	0.604514	157.619	24.0518	7.42354
3  	22    	4.46469	0.0786569	10.4546	2.16965	4.99853
4  	26    	12.0652	0.0557195	419.839	59.0746	1.04479
5  	27    	0.506957	0.0386586	3.46404	0.728329	0.26086
Best individual is: [0.012015262712115471, 0.01400118069072751, 0.8666123177653835]
with fitness: (0.038658563251895504,)
0.018968253604333803
0.018968253604333803
0.018968253604333803
[('0001.HK', Timestamp('2019-09-21 00:00:00'), 208.17897225360434, 208.16000400000001, 0.018968253604333803), ('0001.HK', Timestamp('2019-09-22 00:00:00'), 208.19794050720867, 208.17897225360434, 0.018968253604333803), ('0001.HK', Timestamp('2019-09-23 00:00:00'), 208.216908760813, 208.19794050720867, 0.018968253604333803)]


/Users/shadmanspc/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:94: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
